In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model_name = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B"

In [3]:
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
!nvidia-smi

Sat Jun  7 16:23:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:26:00.0 Off |                  Off |
| 30%   38C    P0             84W /  450W |   16019MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# prepare the model input
# prompt = "Explain O'Reilly online learning"
prompt = "How many 'r's are in 'strawberry'?"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)

In [6]:
# we check what happens when we remove thinking later
text

"<｜begin▁of▁sentence｜><｜User｜>How many 'r's are in 'strawberry'?<｜Assistant｜>"

In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [ ]:
%%time 
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)

In [ ]:
# only read output, skip input
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

In [ ]:
len(output_ids)

In [ ]:
# possibly two different content parts
try:
    # rindex finding 151668 (</think>), could also look for the string!
    think_index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    think_index = 0

thinking_content = tokenizer.decode(output_ids[:think_index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[think_index:], skip_special_tokens=True).strip("\n")

In [ ]:
print(thinking_content)

In [ ]:
print(content)

In [ ]:
from IPython.display import display, Markdown
display(Markdown(content))